## Python Module

Programatic Experiment(Nothing to do with Ananlaysing my Financial Health right now). 

- Extend Data Frame Class as Reporing? 
- Extend Aggregated Data Frame as Class? 

Features: 

- [ ] Assign Category Tag? 
- [ ] Assign Category Tag based on a set of regex match 
- [ ] Assign (in global session) Manual Tags Matched up to ID. 

In [1]:
# import os   
# os.chdir('..')

In [ ]:
from importlib import reload
reload(src.bank_statement_frankliang.analyser)

## Step By Step Break-Down

In [3]:
from src.pdf_bank_statement.reader import BankTransactionProcesser as Processor

processor = Processor()

free_text = processor.extract_text_from_pdf('note/Statement_2023_9.pdf')
relevent_field = processor.extract_transaction_blocks(free_text)
processor.process_transactions(relevent_field)
processor.get_concatenated_dataframe()

,Date,Description,Type,Money In (£),Money Out (£),Balance (£),file
0,01 Sep 23,EXETER CITY COUNCI,DD,blank,5,"5,121",note/Statement_2023_9.pdf
1,01 Sep 23,EXETER CITY COUNCI,DD,blank,29,"5,092",note/Statement_2023_9.pdf
2,01 Sep 23,WATERSTONES,DEB,blank,2,"5,089",note/Statement_2023_9.pdf
3,01 Sep 23,Zettle_*The Explod,DEB,blank,6,"5,082",note/Statement_2023_9.pdf
4,01 Sep 23,MARKS&SPENCER PLC,DEB,blank,13,"5,069",note/Statement_2023_9.pdf
...,...,...,...,...,...,...,...
74,18 Sep 23,BURGER KING,DEB,blank,8,"2,711",note/Statement_2023_9.pdf
75,18 Sep 23,ZETTLE_*130 BASEME,DEB,blank,3,"2,708",note/Statement_2023_9.pdf
76,18 Sep 23,MARKS&SPENCER PLC,DEB,blank,8,"2,700",note/Statement_2023_9.pdf
77,18 Sep 23,TK MAXX,DEB,blank,14,"2,685",note/Statement_2023_9.pdf


## Production 

### Bulk Ingestion

In [3]:
# bulk read (only do this)
import os

statement_files = "/Users/frankliang/Dropbox/Apps/Bank-Statement-Reader/BANK_STATEMENT"
all_files = os.listdir(statement_files)

all_statement=[*map(lambda x: statement_files + '/' + x, all_files)]

from pdf_bank_statement import reader
df = reader.read(all_statement)
df

Read... Total 1 tables populated.
Read... Total 2 tables populated.
Read... Total 3 tables populated.
Read... Total 4 tables populated.
Read... Total 5 tables populated.
Read... Total 6 tables populated.
Read... Total 7 tables populated.
Read... Total 8 tables populated.
Read... Total 9 tables populated.
Read... Total 10 tables populated.
Read... Total 11 tables populated.
cleaning complete.


In [39]:
from src.pdf_bank_statement.analyser import BankTransactionAnalyser as analyser
my_data=analyser(df).data

data_to_ingest = my_data[["Date", "Description", "Type", "Money In (£)","Money Out (£)", "Balance (£)", "Category", "src_file"]]

data is valid
initialized


In [ ]:
import duckdb

DB_PATH = '/Users/frankliang/Dropbox/Apps/DuckDb/personal.db'
with duckdb.connect(DB_PATH) as con:
    con.sql(
        '''
        CREATE TABLE bank_statement as from data_to_ingest
        '''
        )

In [48]:
DB_PATH = '/Users/frankliang/Dropbox/Apps/DuckDb/personal.db'
con = duckdb.connect(DB_PATH) 
con.sql("SELECT * FROM bank_statement").limit(3)

┌─────────────────────┬──────────────────┬─────────┬──────────────┬───┬─────────────┬───────────┬──────────────────────┐
│        Date         │   Description    │  Type   │ Money In (£) │ … │ Balance (£) │ Category  │       src_file       │
│    timestamp_ns     │     varchar      │ varchar │    double    │   │   double    │  varchar  │       varchar        │
├─────────────────────┼──────────────────┼─────────┼──────────────┼───┼─────────────┼───────────┼──────────────────────┤
│ 2023-10-02 00:00:00 │ STGCOACH/CTYLINK │ DEB     │          0.0 │ … │      5770.0 │ Transport │ /Users/frankliang/…  │
│ 2023-10-02 00:00:00 │ PRET A MANGER    │ DEB     │          0.0 │ … │      5764.0 │ Cafe      │ /Users/frankliang/…  │
│ 2023-10-02 00:00:00 │ STGCOACH/CTYLINK │ DEB     │          0.0 │ … │      5762.0 │ Transport │ /Users/frankliang/…  │
├─────────────────────┴──────────────────┴─────────┴──────────────┴───┴─────────────┴───────────┴──────────────────────┤
│ 3 rows                        

## Build in Graphics

In [15]:
my_statement.plotly_spending_period()

In [22]:
BankTransactionAnalyser.darkmode = True # this class is initialised in
my_statement.plotly_spending_period()

In [4]:
my_statement.data.query("Description.str.contains('EXETER LEISURE')")

,Date,Description,Type,Money In (£),Money Out (£),Balance (£),Category,Month,Week


In [17]:
import re
if not bool(re.search('EXETER LEISURE', my_statement.category['Subscri & Apple'])):
    my_statement.category['Subscri & Apple'] += '|EXETER LEISURE' 

In [18]:
my_statement.update_data()
my_statement.data.query("Description.str.contains('EXETER LEISURE')")

,Date,Description,Type,Money In (£),Money Out (£),Balance (£),Category,Month,Week


In [19]:
my_statement2=BankTransactionAnalyser(df)
my_statement2.data.query("Description.str.contains('EXETER LEISURE')")
#my_statement2.plotly_balance() # is still light mode because dark_mode is initialised inside init

data is valid
initialized


,Date,Description,Type,Money In (£),Money Out (£),Balance (£),Category,Month,Week


In [23]:
my_statement2.plotly_balance()

In [21]:
my_statement2.darkmode is my_statement.darkmode

False

Understand Python Class

cleaning complete.


In [48]:
from pdf_bank_statement import reader as bank_reader
from pdf_bank_statement.analyser import BankTransactionAnalyser as bta
import re

test_file_path ='src/resources/Statement_2023_9.pdf'
df = bank_reader.read(test_file_path)
def which_category(x):# process a vender to key based on regex
    category_dict=bta.category
    for category in category_dict:
        match_key = category_dict[category]
        if re.match(match_key, x):
            return(category)
    return('Other')

description = df["Description"]

for value, rex in bta.category.items():
    description.replace(rex, value, regex=True,inplace=True)
description

cleaning complete.


0         Subscri & Apple
1         Subscri & Apple
2             WATERSTONES
3      Zettle_*The Explod
4          Food & Grocery
              ...        
129     Transport/CTYLINK
130                  Cafe
131     Transport/CTYLINK
132        Food & Grocery
133        Food & Grocery
Name: Description, Length: 134, dtype: object

In [30]:
[*df.columns] == ['Date', 'Description', 'Type', 'Money In (£)', 'Money Out (£)',
       'Balance (£)', 'Category', 'src_file']

True

In [13]:
from src.pdf_bank_statement import reader, analyser

df = reader.read('tests/integration/fixtures/lloyd_bank_sample.pdf')
analysed=analyser.analyse(df)

Read... Total 1 tables populated.
cleaning complete.


In [14]:
assert np.array([*analyser.BankTransactionAnalyser.category_maper.keys()]).sort() == analysed.data.Category.unique().sort()

In [17]:
df.Category.unique()

array(['Subscri & Apple', 'Other', 'Food & Grocery', 'Transport', 'Cafe',
       'Rent & Essential'], dtype=object)